In [1]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(
    dbname="recruiting",
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port=6543  # local port you forwarded
)
query = """SELECT
  p.id,
  p."fullName",
  STRING_AGG(DISTINCT COALESCE(e."schoolName", '') || ' (' || COALESCE(e.degree, '') || ' in ' || COALESCE(e."fieldOfStudy", '') || ')', '; ') AS education_summary,
  STRING_AGG(DISTINCT COALESCE(pos.title, '') || ' at ' || COALESCE(pos."companyName", '') || ' in ' || COALESCE(pos.location, ''), '; ') AS position_summary,
  p.summary,
  p.skills,
  p.location,
  p.country,
  p.city
FROM person_data p
LEFT JOIN education_data e ON p.username = e.username
LEFT JOIN position_data pos ON p.username = pos.username
GROUP BY p.id, p."fullName", p.summary, p.skills, p.location, p.country, p.city;

"""

df_raw = pd.read_sql_query(query, conn)

# Create the 'text' column by concatenating all fields into a string
df = df_raw.copy()
df["text"] = df.apply(lambda row: f"""
Full Name: {row['fullName']}.
Education: {row['education_summary']}.
Experience: {row['position_summary']}.
Summary: {row['summary']}.
Skills: {row['skills']}.
Location: {row['location']}, {row['city']}, {row['country']}.
""", axis=1)

df = df[["id", "text"]]
df

OperationalError: connection to server at "127.0.0.1", port 6543 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?


In [45]:
import pandas as pd
import requests
import dotenv

dotenv.load_dotenv()
# Example dataframe

vacancy_text = """
Название должности: Product Manager
Альтернативные тайтлы: Country Manager, Business Development Manager, Regional Manager
Опыт в годах: 3
Обязательные навыки: Product development, Advanced Spanish, Latam region, Native Russian
Дополнительно важно: важен русскоговорящий к-т
Полезные навыки: New markets, Sales management, market research, budgeting
Домены: must have  iGaming, Gambling
Локации поиска: Кипр, Латвия, Грузия, Мальта, Польша, Сербия, Мексика, Испания, Эстония, Россия, Болгария, Литва, Казахстан, Португалия, Таиланд, Бразилия, Аргентина, Чили, Турция
Гражданство: не Армении, не Грузии

"""

# Convert dataframe to list of dicts
candidates = df.to_dict(orient="records")

# Request payload
payload = {
    "vacancy_text": vacancy_text,
    "candidates": candidates
}

# Send POST request
response = requests.post("http://localhost:8000/match_candidates", json=payload)
print(response.text)


{"custom_id":{"0":"1","1":"2","2":"3","3":"4","4":"5","5":"6","6":"7","7":"8","8":"9","9":"10","10":"11","11":"12","12":"13","13":"14","14":"15","15":"16","16":"17","17":"18","18":"19","19":"20","20":"21","21":"22","22":"23","23":"24","24":"25","25":"26","26":"27","27":"28","28":"29","29":"30","30":"31","31":"32","32":"33","33":"34","34":"35","35":"36","36":"37","37":"38","38":"39","39":"40","40":"41","41":"42","42":"43","43":"44","44":"45","45":"46","46":"47","47":"48","48":"49","49":"50","50":"51","51":"52","52":"53","53":"54","54":"55","55":"56","56":"57","57":"58","58":"59","59":"60","60":"61","61":"62","62":"63","63":"64","64":"65","65":"66","66":"67","67":"68","68":"69","69":"70","70":"71","71":"72","72":"73","73":"74","74":"75","75":"76","76":"77","77":"78","78":"79","79":"80","80":"81","81":"82","82":"83","83":"84","84":"85","85":"86","86":"87","87":"88","88":"89","89":"90","90":"91","91":"92","92":"93","93":"94","94":"95","95":"96","96":"97","97":"98","98":"99","99":"100","100

In [46]:
import json
data = json.loads(response.text)

# The response contains two keys: "custom_id" and "content",
# each mapping indices to the appropriate values.
rows = []
# Loop over the sorted keys (as integers) to maintain order.
for idx in sorted(data["custom_id"], key=lambda x: int(x)):
    custom_id = data["custom_id"][idx]
    # The 'content' is itself a JSON string (structured output)
    content_str = data["content"][idx]
    try:
        # Convert the JSON string to an object (dictionary)
        content_obj = json.loads(content_str)
    except json.JSONDecodeError:
        content_obj = content_str
    rows.append({
        "custom_id": custom_id,
        "content": content_obj
    })

# Create DataFrame:
df_results = pd.DataFrame(rows)
df_results

,custom_id,content
0,1,"{'experience': {'feature': 'Experience', 'scor..."
1,2,"{'experience': {'feature': 'Experience', 'scor..."
2,3,"{'experience': {'feature': 'Experience', 'scor..."
3,4,{'experience': {'feature': 'Experience in Prod...
4,5,"{'experience': {'feature': 'Experience', 'scor..."
...,...,...
198,199,{'experience': {'feature': 'Years of Experienc...
199,200,"{'experience': {'feature': 'Experience', 'scor..."
200,201,"{'experience': {'feature': 'Experience', 'scor..."
201,202,"{'experience': {'feature': 'Experience', 'scor..."


In [47]:
#now flatten the content column {'experience': {'feature': 'Experience in product management', 'score': 3, 'explanation': 'The candidate has 3 years of experience, transitioning from a Project Manager to a Product Manager, which closely aligns with the required experience level for the role.'}, 'skills': {'feature': 'Relevant skills for product management', 'score': 3, 'explanation': 'The candidate possesses product development and management skills but lacks proficiency in advanced Spanish and knowledge of Latin American markets, which are essential requirements.'}, 'domain': {'feature': 'Experience in the iGaming or Gambling industry', 'score': 0, 'explanation': 'The candidate does not mention any experience in the iGaming or Gambling domains, which is a necessary criterion for this role.'}, 'location': {'feature': 'Willingness to relocate from Armenia', 'score': 0, 'explanation': 'The candidate is currently based in Armenia and does not meet the citizenship requirement as they are Armenian.'}, 'overall_summary': {'score': 1, 'summary': 'The candidate demonstrates relevant experience and product management skills, but fails to meet critical requirements including domain experience in iGaming/Gambling, advanced Spanish proficiency, and appropriate citizenship for the role.'}}

# with id

df_results = pd.json_normalize(df_results["content"])
df_results["custom_id"] = df_results.index

df_results

,experience.feature,experience.score,experience.explanation,skills.feature,skills.score,skills.explanation,domain.feature,domain.score,domain.explanation,location.feature,location.score,location.explanation,overall_summary.score,overall_summary.summary,custom_id
0,Experience,5,"The candidate has 7 years of experience, which...",Skills,1,The candidate lacks the mandatory skills of ad...,Domain Expertise,1,The candidate does not have experience in iGam...,Location,1,"The candidate is located in Thailand, which is...",2,The candidate shows strong experience but lack...,0
1,Experience,2,The candidate has extensive experience in E-co...,Skills,1,"The candidate lacks advanced Spanish, which is...",Domain Expertise,0,The candidate does not have experience in the ...,Location,0,"The candidate is located in Istanbul, Turkey, ...",0,The candidate does not meet the necessary crit...,1
2,Experience,8,The candidate has over eight years of experien...,Skills,0,The candidate lacks mandatory skills such as a...,Domain Expertise,0,The candidate does not have experience in the ...,Location,0,"The candidate is located in Istanbul, Turkey, ...",3,The candidate has strong experience in UI/UX d...,2
3,Experience in Product Management,3,The candidate has 3 years of experience in pro...,Skills Match,2,The candidate has product development experien...,Domain Expertise,1,The candidate has no explicitly mentioned expe...,Location Suitability,0,"The candidate is located in Yerevan, Armenia, ...",1,The candidate has the required experience for ...,3
4,Experience,2,The candidate has experience in product manage...,Skills,0,The candidate lacks the mandatory skills of 'A...,Domain Expertise,0,The candidate does not have any stated experie...,Location,0,"The candidate is located in Turkey, which is n...",0,The candidate does not meet the essential skil...,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Years of Experience,10,"The candidate has 12 years of experience, sign...",Spanish Language Proficiency and Domain-specif...,0,"The candidate speaks Spanish, but lacks critic...",Domain Experience in iGaming and Gambling,0,The candidate does not have experience in iGam...,Geographical Location,0,"The candidate is located in São Paulo, Brazil,...",2,"Overall, the candidate has substantial experie...",198
199,Experience,3,The candidate has several years of experience ...,Skills,0,The candidate does not meet the mandatory skil...,Domain expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Brazil, which is n...",1,The candidate has relevant experience but fail...,199
200,Experience,1,The candidate has extensive experience in prod...,Skills,0,The candidate does not meet the mandatory skil...,Domain Expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Turkey, which is n...",0,The candidate does not meet several critical r...,200
201,Experience,2,The candidate has several years of experience ...,Skills,1,The candidate possesses some relevant skills i...,Domain Expertise,0,The candidate does not have any experience in ...,Location,0,"The candidate is located in Turkey, which is n...",1,The candidate does not meet the critical requi...,201


In [48]:
# str id
df["id"] = df.index
df["id"] = df["id"].astype(str)
df

,id,text
0,0,\nFull Name: Ayush Jha.\nEducation: Birla Inst...
1,1,\nFull Name: Buket Comert.\nEducation: Boğaziç...
2,2,\nFull Name: Ahmad Sekmani.\nEducation: The Fo...
3,3,\nFull Name: Arpine Lputyan.\nEducation: Frenc...
4,4,\nFull Name: Alper Özer.\nEducation: Izmir Uni...
...,...,...
198,198,\nFull Name: Mat Silva.\nEducation: Cursos PM3...
199,199,\nFull Name: Beatriz Bottino.\nEducation: Univ...
200,200,\nFull Name: Iman Behboudi.\nEducation: comput...
201,201,\nFull Name: DUYGU KÖPÜKLÜ.\nEducation: Anadol...


In [49]:

df_results["custom_id"] = df_results["custom_id"].astype(str)
#merge with df
df_results = df_results.merge(df, left_on="custom_id", right_on="id", how="left")
df_results = df_results.drop(columns=["custom_id", "id"])
df_results

,experience.feature,experience.score,experience.explanation,skills.feature,skills.score,skills.explanation,domain.feature,domain.score,domain.explanation,location.feature,location.score,location.explanation,overall_summary.score,overall_summary.summary,text
0,Experience,5,"The candidate has 7 years of experience, which...",Skills,1,The candidate lacks the mandatory skills of ad...,Domain Expertise,1,The candidate does not have experience in iGam...,Location,1,"The candidate is located in Thailand, which is...",2,The candidate shows strong experience but lack...,\nFull Name: Ayush Jha.\nEducation: Birla Inst...
1,Experience,2,The candidate has extensive experience in E-co...,Skills,1,"The candidate lacks advanced Spanish, which is...",Domain Expertise,0,The candidate does not have experience in the ...,Location,0,"The candidate is located in Istanbul, Turkey, ...",0,The candidate does not meet the necessary crit...,\nFull Name: Buket Comert.\nEducation: Boğaziç...
2,Experience,8,The candidate has over eight years of experien...,Skills,0,The candidate lacks mandatory skills such as a...,Domain Expertise,0,The candidate does not have experience in the ...,Location,0,"The candidate is located in Istanbul, Turkey, ...",3,The candidate has strong experience in UI/UX d...,\nFull Name: Ahmad Sekmani.\nEducation: The Fo...
3,Experience in Product Management,3,The candidate has 3 years of experience in pro...,Skills Match,2,The candidate has product development experien...,Domain Expertise,1,The candidate has no explicitly mentioned expe...,Location Suitability,0,"The candidate is located in Yerevan, Armenia, ...",1,The candidate has the required experience for ...,\nFull Name: Arpine Lputyan.\nEducation: Frenc...
4,Experience,2,The candidate has experience in product manage...,Skills,0,The candidate lacks the mandatory skills of 'A...,Domain Expertise,0,The candidate does not have any stated experie...,Location,0,"The candidate is located in Turkey, which is n...",0,The candidate does not meet the essential skil...,\nFull Name: Alper Özer.\nEducation: Izmir Uni...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,Years of Experience,10,"The candidate has 12 years of experience, sign...",Spanish Language Proficiency and Domain-specif...,0,"The candidate speaks Spanish, but lacks critic...",Domain Experience in iGaming and Gambling,0,The candidate does not have experience in iGam...,Geographical Location,0,"The candidate is located in São Paulo, Brazil,...",2,"Overall, the candidate has substantial experie...",\nFull Name: Mat Silva.\nEducation: Cursos PM3...
199,Experience,3,The candidate has several years of experience ...,Skills,0,The candidate does not meet the mandatory skil...,Domain expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Brazil, which is n...",1,The candidate has relevant experience but fail...,\nFull Name: Beatriz Bottino.\nEducation: Univ...
200,Experience,1,The candidate has extensive experience in prod...,Skills,0,The candidate does not meet the mandatory skil...,Domain Expertise,0,The candidate lacks experience in the iGaming ...,Location,0,"The candidate is located in Turkey, which is n...",0,The candidate does not meet several critical r...,\nFull Name: Iman Behboudi.\nEducation: comput...
201,Experience,2,The candidate has several years of experience ...,Skills,1,The candidate possesses some relevant skills i...,Domain Expertise,0,The candidate does not have any experience in ...,Location,0,"The candidate is located in Turkey, which is n...",1,The candidate does not meet the critical requi...,\nFull Name: DUYGU KÖPÜKLÜ.\nEducation: Anadol...


In [50]:
df_results.to_csv("results.csv", index=False)